In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dense
from keras.layers import GlobalMaxPool1D, Conv1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("./data.csv")

In [4]:
data = data[data.sentiment == 0]

In [5]:
import re
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [6]:
labels = data['sentiment']

In [7]:
x = []
sentences = list(data["reviews"])
for sen in sentences:
    x.append(preprocess_text(sen))

y = labels.values

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10)

In [9]:
tokenizer = Tokenizer(num_words=10000)    # This will limit the maximum number of words to 10000
tokenizer.fit_on_texts(x)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
x = tokenizer.texts_to_sequences(x)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 100      # Maximum length of sequence is capped at 100


# Padding the sequences to match the length

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)
x = pad_sequences(x, padding='post', maxlen=maxlen)

In [10]:
len(x_train[4])

100

In [16]:
import tensorflow as tf

# Set the random seed for reproducibility
tf.random.set_seed(1234)

# Set the dimensions of the input and output data
input_dim = 10000
output_dim = 1000

# Define the generator model
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Embedding(input_dim=input_dim, output_dim=128))
generator.add(tf.keras.layers.LSTM(128))
generator.add(tf.keras.layers.Dense(output_dim, activation='softmax'))

# Define the discriminator model
discriminator = tf.keras.Sequential()
discriminator.add(tf.keras.layers.Embedding(input_dim=output_dim, output_dim=128))
discriminator.add(tf.keras.layers.LSTM(128))
discriminator.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Define the GAN model
gan = tf.keras.Sequential()
gan.add(generator)
gan.add(discriminator)

# Compile the GAN model
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Train the GAN model on your dataset
gan.fit(x_train, y_train, epochs=1)

1359/1359 [==============================] - 1213s 892ms/step - loss: 0.0028


In [24]:
z = tf.random.normal(shape=(1,10), mean=50, stddev=10)
# x = tf.reshape(x_test[0], shape=(1, input_dim))
# Use the GAN to generate new samples of data
generated_data = gan.predict(z)

# Save the generated samples to a file
# with open('generated_data.txt', 'w') as f:
#   for sample in generated_data:
#     f.write(sample + '\n')

generated_data

array([[3.135518e-06]], dtype=float32)

In [25]:
import numpy as np

In [26]:
z = np.array(z).astype(int)

In [27]:
z

array([[48, 62, 50, 47, 39, 42, 64, 60, 43, 61]])

In [31]:
z = np.reshape(z, 10)

In [32]:
z

array([48, 62, 50, 47, 39, 42, 64, 60, 43, 61])

In [33]:
tokenizer.sequences_to_texts([z])

['no great should can so me week one i learning']

In [45]:
new_data = []
for i in range(10000):
    z = tf.random.normal(shape=(1,10), mean=5000, stddev=1000)
    z = np.array(z).astype(int)
    z = np.reshape(z, 10)
    new_data.append(tokenizer.sequences_to_texts([z]))

In [46]:
len(new_data)

10000

In [48]:
data_to_append = []
for i in new_data:
    data_to_append.append(i[0])

In [49]:
data_to_append

['tossed resultados routing downhill axis narration dito elevado principals letting',
 'region apt crashes flight transmitir leadership writers grey hack female',
 'specifics redacci aburrido rooms desarrollar capacity supports innovative canadian eliminate',
 'weaknesses dissapointed ca didactics marker viewed ellas advances erroneous llamar',
 'yaakov qualidade fader asi conocer figures artistic hora genetics assn',
 'reduces acess revisions recommened claim theoritical auditor instagram switching hall',
 'ps bummer upgrading beautifully behavioural fleshed lawyer participaci inevitably mes',
 'cuestionario cross composed audiences raise guessed ao union contenu citing',
 'manually backwards combinations prove manufacturing problema physics indepth ensemble filmed',
 'sees operate proud asset libro flying anza executives programar graduated',
 'arent wird entirety suit skilled resolving authority bite ayudar reinforcement',
 'private navigating lays ingles color ein intonation perd c

In [50]:
data.head()

,reviews,sentiment
81970,Probably the best financial market course on c...,0
81971,"From a knowledge perspective, I have learned a...",0
81972,Some parts of the course were informative. So...,0
81973,"The course is not that challenging, it could b...",0
81974,It's more of a course for the people who have ...,0


In [77]:
aug_data = pd.DataFrame(data_to_append)

In [78]:
aug_data.head()

,0
0,tossed resultados routing downhill axis narrat...
1,region apt crashes flight transmitir leadershi...
2,specifics redacci aburrido rooms desarrollar c...
3,weaknesses dissapointed ca didactics marker vi...
4,yaakov qualidade fader asi conocer figures art...


In [84]:
total_data = data.append(aug_data)
total_data = total_data[["reviews", "sentiment"]]

<ipython-input-84-98a1ab77489f>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_data = data.append(aug_data)


In [85]:
total_data.head()

,reviews,sentiment
81970,Probably the best financial market course on c...,0.0
81971,"From a knowledge perspective, I have learned a...",0.0
81972,Some parts of the course were informative. So...,0.0
81973,"The course is not that challenging, it could b...",0.0
81974,It's more of a course for the people who have ...,0.0


In [87]:
total_data.shape

(58292, 2)

In [91]:
og_data = pd.read_csv("./og_data.csv")

In [92]:
# Function to map stars to sentiment
def map_sentiment(stars_received):
    if stars_received <= 2:
        return -1
    elif stars_received == 3:
        return 0
    else:
        return 1
# Mapping stars to sentiment into three categories
og_data['sentiment'] = [map_sentiment(x) for x in og_data['rating']]

In [99]:
pos_data = og_data[og_data.sentiment==1]

In [100]:
pos_data.shape

(989056, 3)

In [101]:
neg_data = og_data[og_data.sentiment==-1]

In [102]:
req_neg_data = pos_data.shape[0] - neg_data.shape[0]

In [103]:
req_neg_data

965415

In [104]:
new_data = []
for i in range(req_neg_data):
    z = tf.random.normal(shape=(1,10), mean=5000, stddev=2500)
    z = np.array(z).astype(int)
    z = np.reshape(z, 10)
    new_data.append(tokenizer.sequences_to_texts([z]))

In [105]:
data_to_append = []
for i in new_data:
    data_to_append.append(i[0])

In [106]:
aug_data = pd.DataFrame(data_to_append)

In [116]:
total_data = og_data.append(aug_data)
total_data = total_data[["reviews", "sentiment"]]
total_data = total_data.dropna()

<ipython-input-116-f91ede5a605d>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_data = og_data.append(aug_data)


In [117]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(total_data['reviews'], total_data['sentiment'], test_size = 0.3)

In [118]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern= r'\b\w+\b')
train_matrix = vectorizer.fit_transform(x_train)
test_matrix = vectorizer.transform(x_test)

In [119]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [121]:
rf.fit(train_matrix, y_train)

RandomForestClassifier()

In [123]:
preds = rf.predict(test_matrix)

In [124]:
from sklearn.metrics import classification_report
print(classification_report(preds,y_test))

              precision    recall  f1-score   support

        -1.0       0.85      0.99      0.91      6109
         0.0       0.83      0.98      0.90      9134
         1.0       1.00      0.99      1.00    299298

    accuracy                           0.99    314541
   macro avg       0.89      0.99      0.94    314541
weighted avg       0.99      0.99      0.99    314541



In [125]:
import pickle as pkl

In [126]:
with open("model_gan.pkl", "wb") as file:
    pkl.dump(rf, file)

In [127]:
with open("vectorizer_gan.pkl", "wb") as file:
    pkl.dump(vectorizer, file)

In [128]:
gan.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_15 (Sequential)  (None, 1000)              1540584   
                                                                 
 sequential_16 (Sequential)  (None, 1)                 259713    
                                                                 
Total params: 1,800,297
Trainable params: 1,800,297
Non-trainable params: 0
_________________________________________________________________


In [129]:
generator.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, None, 128)         1280000   
                                                                 
 lstm_10 (LSTM)              (None, 128)               131584    
                                                                 
 dense_14 (Dense)            (None, 1000)              129000    
                                                                 
Total params: 1,540,584
Trainable params: 1,540,584
Non-trainable params: 0
_________________________________________________________________


In [130]:
discriminator.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, None, 128)         128000    
                                                                 
 lstm_11 (LSTM)              (None, 128)               131584    
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 259,713
Trainable params: 259,713
Non-trainable params: 0
_________________________________________________________________
